<a href="https://colab.research.google.com/github/krisana-y/229352-StatisticalLearning/blob/main/Lab04_Naive_Bayes_Grid_and_Random_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #4

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

from scipy.stats import uniform

In [ ]:
train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Xtrain = train.data[:3000]
ytrain = train.target[:3000]
Xtest = test.data[:500]
ytest = test.target[:500]

print("X:", len(Xtest))
print("y:", len(ytest))

X: 500
y: 500


### Naive Bayes [(Documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB(alpha=0.1)

### Random Search Cross-Validation [(Documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

### Uniform distribution in `Scipy` [(Documentation)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

Pipeline = Pipeline([('count', CountVectorizer(stop_words="english")),
                     ('nb', MultinomialNB())])

parameter = {'nb__alpha': uniform(loc=0.1,scale=10)}

clf = RandomizedSearchCV(Pipeline, parameter, n_iter=10, cv=3)

clf.fit(Xtrain, ytrain)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('count',
                                              CountVectorizer(stop_words='english')),
                                             ('nb', MultinomialNB())]),
                   param_distributions={'nb__alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x790445f03610>})

In [ ]:
ypredi = clf.predict(Xtest)
print(classification_report(ytest, ypredi))

              precision    recall  f1-score   support

           0       0.74      0.67      0.70        21
           1       0.67      0.48      0.56        21
           2       1.00      0.04      0.07        26
           3       0.51      0.71      0.59        34
           4       0.78      0.85      0.82        34
           5       0.71      0.77      0.74        26
           6       0.86      0.82      0.84        22
           7       0.77      0.96      0.86        28
           8       0.93      0.85      0.89        33
           9       0.91      0.84      0.88        25
          10       0.93      0.96      0.95        27
          11       0.78      0.90      0.84        20
          12       0.57      0.67      0.62        24
          13       0.86      0.83      0.84        23
          14       0.79      0.93      0.85        28
          15       0.77      0.93      0.84        29
          16       0.54      0.95      0.69        21
          17       0.94    

#### Exercise

1. For the Naive Bayes model, use grid search 5-fold cross-validation across different values of `alpha` to find the best model.

2. For the best value of `alpha`, compute the `f1_macro` score on the test set.
* What value of `alpha` did you obtain?
* What is the model's `f1_macro` score?

3. Repeat Exercise 1 and 2 for **random search** 5-fold cross validation across different values of `alpha`. Compute the `f1_macro` score on the test set.
* What value of `alpha` did you obtain?
* Did you get a better `f1_macro` score compared to grid search in Exercise 2?

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

# Exercise 1 & 2: Grid Search
param_grid = {'nb__alpha': [0.01, 0.1, 0.5, 1.0, 5.0, 10.0]}
kf = KFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(Pipeline, param_grid, cv=kf, scoring='f1_macro')
grid_search.fit(Xtrain, ytrain)

best_alpha_grid = grid_search.best_params_['nb__alpha']
print(f"Best alpha from Grid Search: {best_alpha_grid}")

best_grid_model = grid_search.best_estimator_
y_pred_grid = best_grid_model.predict(Xtest)
f1_grid = f1_score(ytest, y_pred_grid, average='macro')
print(f"f1_macro score with Grid Search: {f1_grid}")

# Exercise 3: Random Search
param_dist = {'nb__alpha': uniform(loc=0.01, scale=10)}

random_search = RandomizedSearchCV(Pipeline, param_distributions=param_dist, n_iter=10, cv=kf, scoring='f1_macro', random_state=42)
random_search.fit(Xtrain, ytrain)

best_alpha_random = random_search.best_params_['nb__alpha']
print(f"\nBest alpha from Random Search: {best_alpha_random}")

best_random_model = random_search.best_estimator_
y_pred_random = best_random_model.predict(Xtest)
f1_random = f1_score(ytest, y_pred_random, average='macro')
print(f"f1_macro score with Random Search: {f1_random}")

print(f"\nDid Random Search get a better f1_macro score than Grid Search? {f1_random > f1_grid}")

Best alpha from Grid Search: 0.1
f1_macro score with Grid Search: 0.7188482458540597

Best alpha from Random Search: 0.5908361216819946
f1_macro score with Random Search: 0.7037499416389534

Did Random Search get a better f1_macro score than Grid Search? False
